In [75]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras 
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers.embeddings import Embedding
from keras import regularizers
import emoji
import re
import string
import nltk
from nltk.corpus import stopwords
from sklearn.preprocessing import OneHotEncoder
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import math
import pickle

In [76]:
train = pd.read_csv('movie_reviews.csv')
train.head()

,label,clean_text
0,0,grew b NUMBER watching loving thunderbird mate...
1,0,put movie dvd player sat coke chip expectation...
2,0,people know particular time past like feel nee...
3,0,even though great interest biblical movie bore...
4,1,im die hard dad army fan nothing ever change g...


In [77]:
train.tail()

,label,clean_text
39995,1,western union something forgotten classic west...
39996,1,movie incredible piece work explores every noo...
39997,0,wife watched movie plan visit sicily stromboli...
39998,1,first watched flatliners amazed necessary feat...
39999,1,would film good gross estimated NUMBER award n...


In [78]:
y_train = train['label']

In [79]:
train = train.drop('label',axis=1)

In [80]:
train.head()

,clean_text
0,grew b NUMBER watching loving thunderbird mate...
1,put movie dvd player sat coke chip expectation...
2,people know particular time past like feel nee...
3,even though great interest biblical movie bore...
4,im die hard dad army fan nothing ever change g...


In [81]:
sentences = train['clean_text'].tolist()

labels = y_train.values.tolist()

In [82]:
sentences = [str(i) for i in sentences]

In [83]:
ma = 0
for i in sentences:
    if len(i)>ma:
        ma = len(i) 
        
print(ma)

9330


In [84]:
vocab = 50000
oov = '<OOV>'
embedding = 500
padding = 'post'
truncate = 'post'
maxlength = 5000

In [85]:
ratio = 0.8*len(sentences)
ratio = int(ratio)
# print(ratio)
train = sentences[0:ratio]
train_label = labels[0:ratio]
val = sentences[ratio:]
val_labels = labels[ratio:]

In [86]:
tokenizer =Tokenizer(num_words = vocab, oov_token=oov)
tokenizer.fit_on_texts(train)
word_index = tokenizer.word_index
training = tokenizer.texts_to_sequences(train)
training_pad = pad_sequences(training, maxlen=maxlength, padding=padding, truncating=truncate)

validation = tokenizer.texts_to_sequences(val)
validation_pad = pad_sequences(validation, maxlen=maxlength, padding=padding, truncating=truncate)

In [68]:
tf.keras.backend.clear_session()
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab, embedding, input_length=maxlength),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(300,activation='sigmoid'),
    tf.keras.layers.Dense(150, activation='sigmoid'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])
model.compile(loss='binary_crossentropy',optimizer='adamax',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 5000, 500)         25000000  
                                                                 
 global_average_pooling1d (G  (None, 500)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 300)               150300    
                                                                 
 dense_1 (Dense)             (None, 150)               45150     
                                                                 
 dense_2 (Dense)             (None, 1)                 151       
                                                                 
Total params: 25,195,601
Trainable params: 25,195,601
Non-trainable params: 0
____________________________________________

In [69]:
val_labels = np.array(val_labels)
train_label = np.array(train_label)

In [70]:
num_epochs = 25
history = model.fit(training_pad, 
                    train_label, 
                    epochs=num_epochs, 
                    validation_data=(validation_pad, val_labels))

Epoch 1/25
1000/1000 [==============================] - 277s 276ms/step - loss: 0.6967 - accuracy: 0.4987 - val_loss: 0.6948 - val_accuracy: 0.4983
Epoch 2/25
1000/1000 [==============================] - 277s 277ms/step - loss: 0.6945 - accuracy: 0.5097 - val_loss: 0.6960 - val_accuracy: 0.5017
Epoch 3/25
1000/1000 [==============================] - 277s 277ms/step - loss: 0.6773 - accuracy: 0.5769 - val_loss: 0.6431 - val_accuracy: 0.7028
Epoch 4/25
1000/1000 [==============================] - 274s 274ms/step - loss: 0.5585 - accuracy: 0.7566 - val_loss: 0.4680 - val_accuracy: 0.8195
Epoch 5/25
1000/1000 [==============================] - 271s 271ms/step - loss: 0.4280 - accuracy: 0.8177 - val_loss: 0.3946 - val_accuracy: 0.8384
Epoch 6/25
1000/1000 [==============================] - 272s 272ms/step - loss: 0.3772 - accuracy: 0.8396 - val_loss: 0.3586 - val_accuracy: 0.8596
Epoch 7/25
1000/1000 [==============================] - 275s 275ms/step - loss: 0.3494 - accuracy: 0.8530 - val_

In [95]:
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [199]:
model.save(r'C:\Users\robtu')

INFO:tensorflow:Assets written to: C:\Users\robtu\assets
